# Python Logging in Klassen

Da wir Objektorientiert Arbeiten, sollten wir nicht in jeder Klasse einen Logger initialisieren, sondern den Logger in eine eigene Klasse auslagern, denn wir wollen doppelten Code vermeiden!

In [1]:
import logging
from logging.handlers import RotatingFileHandler

def my_logger(name):
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    
    formatter = logging.Formatter("%(levelname)s %(asctime)s: \n %(message)s", "%d.%m.%Y %H:%M:%S")
    file_handler = RotatingFileHandler(filename=f"../log/{name}.log", maxBytes=1024000, backupCount=3)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    return logger

In [3]:
from helper import DbMock
import time

class Calculator:
    def __init__(self, divident, divisor):
        self.logger = my_logger(__class__.__name__)
        self.divident = divident
        self.divisor = divisor

    def run(self):
        self._divide()
           
    def _divide(self):
        try:
            self.logger.info(f"divide {self.divident} / {self.divisor}")
            quotient = self.divident / self.divisor
            self.logger.info(f"{self.divident} / {self.divisor} = {quotient}")
            return quotient
        except ZeroDivisionError as ex:
            self.logger.error(f"beim teilen von ({self.divident} / {self.divisor}) ist ein fehler aufgetreten:", exc_info=True)
           
    def __del__(self):
        for handler in self.logger.handlers[:]:
            self.logger.removeHandler(handler)
        print(f"{__class__.__name__} wurde Zerstört")


class SQLConnection:
    def __init__(self, url):
        self.logger = my_logger(__class__.__name__)
        self.url = url

    def run(self):
        self._connect()

    def _connect(self):
        try:
            DbMock.connect(self.url)
        except Exception as ex:
            self.logger.error(ex, exc_info=True)
       
    def __del__(self):
        for handler in self.logger.handlers[:]:
            self.logger.removeHandler(handler)
        print(f"{__class__.__name__} wurde Zerstört")

calc = Calculator(20,0)
sql = SQLConnection(1)

calc.run()
sql.run()


beim teilen von (20 / 0) ist ein fehler aufgetreten:
Traceback (most recent call last):
  File "C:\Users\MEINES~1\AppData\Local\Temp/ipykernel_17272/1214196708.py", line 16, in _divide
    quotient = self.divident / self.divisor
ZeroDivisionError: division by zero

Traceback (most recent call last):
  File "C:\Users\MEINES~1\AppData\Local\Temp/ipykernel_17272/1214196708.py", line 38, in _connect
    DbMock.connect(self.url)
  File "c:\own_projects\python-Kurs\Diverses\jupyter\helper.py", line 31, in connect
    raise TypeError()
TypeError


Calculator wurde Zerstört
SQLConnection wurde Zerstört
Calculator wurde Zerstört
SQLConnection wurde Zerstört


In [ ]:
from helper import DbMock
import threading
import time

class Calculator(threading.Thread):
    def __init__(self, divident, divisor):
        super(Calculator, self).__init__()
        self.logger = my_logger(__class__.__name__)
        self._stop = threading.Event()
        self.divident = divident
        self.divisor = divisor

    def stop(self):
        self._stop.set()

    def _stopped(self):
        return self._stop.is_set()

    def run(self):
        while True:
            self._divide()
            if self._stopped():
                return

    def _divide(self):
        try:
            self.logger.info(f"divide {self.divident} / {self.divisor}")
            quotient = self.divident / self.divisor
            self.logger.info(f"{self.divident} / {self.divisor} = {quotient}")
            return quotient
        except ZeroDivisionError as ex:
            self.logger.error(f"beim teilen von ({self.divident} / {self.divisor}) ist ein fehler aufgetreten:", exc_info=True)
           
    def __del__(self):
        for handler in self.logger.handlers[:]:
            self.logger.removeHandler(handler)
        print(f"{__class__.__name__} wurde Zerstört")


class SQLConnection(threading.Thread):
    def __init__(self, url):
        super(SQLConnection, self).__init__()
        self.logger = my_logger(__class__.__name__)
        self._stop = threading.Event()
        self.url = url

    def stop(self):
        self._stop.set()

    def _stopped(self):
        return self._stop.is_set()

    def run(self):
        while True:
            self._connect()
            if self._stopped():
                return

    def _connect(self):
        try:
            DbMock.connect(self.url)
        except Exception as ex:
            self.logger.error(ex, exc_info=True)
       
    def __del__(self):
        for handler in self.logger.handlers[:]:
            self.logger.removeHandler(handler)
        print(f"{__class__.__name__} wurde Zerstört")

thread_1 = Calculator(20,0)
thread_2 = SQLConnection(1)

thread_1.start()
thread_2.start()
time.sleep(60)
print("stopping threads")
thread_1.stop()
thread_2.stop()
time.sleep(2)
thread_1.__del__()
thread_2.__del__()
